In [1]:
import pandas as pd
import numpy as np 
import datetime as dt
from datetime import timedelta
import matplotlib as plt

In [2]:
Top = pd.read_excel('WCRI Total Claim Cost 12_24_36 2000-2010.xlsx')
Middle = pd.read_excel('WCRI Total Claim Cost 12_24_36 2011_2017.xlsx')
Bottom = pd.read_excel('WCRI Total Claim Cost 12_24_36 2018_2023.xlsx')
twelve = pd.concat([Top, Middle, Bottom])
twentyfour = pd.concat([Top, Middle, Bottom])
thirtysix = pd.concat([Top, Middle, Bottom])

In [ ]:
def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

# Reference table for incident date ranges and maturity dates
reference_table = {
    ("2016-10-01", "2017-09-30"): {"12 Month Maturity": "2018-03-31", "24 Month Maturity": "2018-03-31"},
    ("2017-10-01", "2018-09-30"): {"12 Month Maturity": "2019-03-31", "24 Month Maturity": "2020-03-31"},
    ("2018-10-01", "2019-09-30"): {"12 Month Maturity": "2020-03-31", "24 Month Maturity": "2021-03-31"}
}

# Function to calculate maturity date based on incident date
def calculate_maturity_date(incident_date, maturity_date_str):
    maturity_date = parse_date(maturity_date_str)
    return maturity_date

# Example incident date
incident_date_str = "2017-12-15"
incident_date = parse_date(incident_date_str)

# Lookup maturity dates
for date_range, maturity_dates in reference_table.items():
    start_date, end_date = parse_date(date_range[0]), parse_date(date_range[1])
    if start_date <= incident_date <= end_date:
        maturity_12_month = calculate_maturity_date(incident_date, maturity_dates["12 Month Maturity"])
        maturity_24_month = calculate_maturity_date(incident_date, maturity_dates["24 Month Maturity"])
        print("Transaction date range (12 Month Maturity):", incident_date, "to", maturity_12_month)
        print("Transaction date range (24 Month Maturity):", incident_date, "to", maturity_24_month)
        break
else:
    print("Incident date not found in reference table.")


In [ ]:
#Convert "Date of Loss/Injury" column to datetime format
df['Date of Loss/Incident'] = pd.to_datetime(df['Date of Incident/Loss'])

#Create a new column for fiscal year
df['Fiscal Year'] = df['Date of Loss/Injury'].dt.year
df.loc[(df['Date of Loss/Injury'].dt.month >= 10), 'Fiscal Year'] += 1

#Filter DataFrame based on fiscal year and 12-month maturity mark
df_filtered = df[(df['Date of Loss/Injury'] >= pd.to_datetime(df['Fiscal Year'], format='%Y') - pd.DateOffset(years=1)) & 
                 (df['Date of Loss/Injury'] <= pd.to_datetime(df['Fiscal Year'], format='%Y') + pd.DateOffset(months=6) - pd.DateOffset(days=1))]

# Now df_filtered contains transactions at the 12-month maturity mark


In [16]:
def calculate_fiscal_year(incident_date):
    fiscal_year_start_month = 10  # Fiscal year starts in October
    if incident_date.month >= fiscal_year_start_month:
        return incident_date.year + 1
    else:
        return incident_date.year
    
twelve['Fiscal Year'] = twelve['Date of Incident/Loss'].apply(calculate_fiscal_year)

In [20]:
twelve['Date of Incident/Loss'] = pd.to_datetime(twelve['Date of Incident/Loss'])
twelve['Transaction Date'] = pd.to_datetime(twelve['Transaction Date'])

twelve['Year of Incident'] = twelve['Date of Incident/Loss'].dt.year

# Define the function to sum transactions based on specified periods and bucket by year of incident
def sum_transactions(df, start_date, end_date, valuation_date):
    """
    Sums up transactions for each Claim Number within the specified period and adds the year of incident.

    Parameters:
    df (pd.DataFrame): DataFrame containing claim transactions.
    start_date (str): Start date for Date of Loss/Incident range.
    end_date (str): End date for Date of Loss/Incident range.
    valuation_date (str): Valuation date until which to sum transactions.

    Returns:
    pd.DataFrame: DataFrame with Claim Number, Year of Incident, and their respective summed amounts.
    """
    # Filter claims within the date range
    filtered_claims = df[(df['Date of Incident/Loss'] >= start_date) & 
                         (df['Date of Incident/Loss'] <= end_date)]
    
    # Further filter transactions up to the valuation date
    filtered_transactions = filtered_claims[filtered_claims['Transaction Date'] <= valuation_date]
    
    # Sum the transactions for each claim number and year of incident
    summed_transactions = filtered_transactions.groupby(['Claim Number', 'Fiscal Year'])['Paid Amount'].sum().reset_index()
    summed_transactions.rename(columns={'Paid Amount': 'Total Amount'}, inplace=True)
    
    return summed_transactions

# Define date ranges and corresponding valuation dates
date_ranges = [
    {'start_date': '2000-10-01', 'end_date': '2001-09-30', 'valuation_date': '2002-03-31'},
    {'start_date': '2001-10-01', 'end_date': '2002-09-30', 'valuation_date': '2003-03-31'},
    {'start_date': '2002-10-01', 'end_date': '2003-09-30', 'valuation_date': '2004-03-31'},
    {'start_date': '2003-10-01', 'end_date': '2004-09-30', 'valuation_date': '2005-03-31'},
    {'start_date': '2004-10-01', 'end_date': '2005-09-30', 'valuation_date': '2006-03-31'},
    {'start_date': '2005-10-01', 'end_date': '2006-09-30', 'valuation_date': '2007-03-31'},
    {'start_date': '2006-10-01', 'end_date': '2007-09-30', 'valuation_date': '2008-03-31'},
    {'start_date': '2007-10-01', 'end_date': '2008-09-30', 'valuation_date': '2009-03-31'},
    {'start_date': '2008-10-01', 'end_date': '2009-09-30', 'valuation_date': '2010-03-31'},
    {'start_date': '2009-10-01', 'end_date': '2010-09-30', 'valuation_date': '2011-03-31'},
    {'start_date': '2010-10-01', 'end_date': '2011-09-30', 'valuation_date': '2012-03-31'},
    {'start_date': '2011-10-01', 'end_date': '2012-09-30', 'valuation_date': '2013-03-31'},
    {'start_date': '2012-10-01', 'end_date': '2013-09-30', 'valuation_date': '2014-03-31'},
    {'start_date': '2013-10-01', 'end_date': '2014-09-30', 'valuation_date': '2015-03-31'},
    {'start_date': '2014-10-01', 'end_date': '2015-09-30', 'valuation_date': '2016-03-31'},
    {'start_date': '2015-10-01', 'end_date': '2016-09-30', 'valuation_date': '2017-03-31'},
    {'start_date': '2016-10-01', 'end_date': '2017-09-30', 'valuation_date': '2018-03-31'},
    {'start_date': '2017-10-01', 'end_date': '2018-09-30', 'valuation_date': '2019-03-31'},
    {'start_date': '2018-10-01', 'end_date': '2019-09-30', 'valuation_date': '2020-03-31'},
    {'start_date': '2019-10-01', 'end_date': '2020-09-30', 'valuation_date': '2021-03-31'},
    {'start_date': '2020-10-01', 'end_date': '2021-09-30', 'valuation_date': '2022-03-31'},
    {'start_date': '2021-10-01', 'end_date': '2022-09-30', 'valuation_date': '2023-03-31'},
    {'start_date': '2022-10-01', 'end_date': '2023-09-30', 'valuation_date': '2024-03-31'},
    {'start_date': '2023-10-01', 'end_date': '2024-09-30', 'valuation_date': '2025-03-31'},
    # Add more ranges as needed up to 2023
]

# Initialize an empty DataFrame to store results
all_summed_transactions = pd.DataFrame()

# Iterate over date ranges and apply the function
for date_range in date_ranges:
    summed_transactions = sum_transactions(
        twelve, 
        date_range['start_date'], 
        date_range['end_date'], 
        date_range['valuation_date']
    )
    all_summed_transactions = pd.concat([all_summed_transactions, summed_transactions], ignore_index=True)

print(all_summed_transactions)


         Claim Number  Fiscal Year  Total Amount
0          W200001719         2000       4207.63
1          W200001956         2000      21561.39
2          W200003311         2000      13653.71
3          W200003479         2000      19215.99
4          W200003906         2000      42165.56
...               ...          ...           ...
16828  WDWW2023231479         2023       5375.15
16829  WDWW2023231549         2023       7007.03
16830  WDWW2023231797         2023       4615.37
16831  WDWW2023232702         2023       6338.44
16832  WDWW2023235604         2023       5524.16

[16833 rows x 3 columns]


In [21]:
all_summed_transactions.to_csv('J:/Data/RMSA Analysis/0 - Team Working Files/10 - Nick/WCRIv412month.csv', index=False)